# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf "/content/drive/MyDrive/EECS592/dev.tar" -C "/content"
!tar -xf "/content/drive/MyDrive/EECS592/test.tar" -C "/content"
!tar -xf "/content/drive/MyDrive/EECS592/train.tar" -C "/content"
!mkdir images

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'


# Dataloader

In [ ]:
from transformers import CLIPProcessor, CLIPModel

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F

from tqdm import tqdm, trange


In [ ]:
device = 'cpu'
if torch.backends.mps.is_available():
    device = 'mps'
if torch.cuda.is_available():
    device = 'cuda'
print(f"Using '{device}' device")

Using 'cuda' device


In [ ]:
from transformers import CLIPProcessor, CLIPModel


def get_clip_features(image, preprocess, model, device):
    image = preprocess(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        features = model.get_image_features(**image)
    return features.squeeze(0)


In [ ]:

class MIT5KDataset(Dataset):
    def __init__(self, root_dir, transform=None, clip_model=None, preprocess=None, device=None):
        self.root_dir = root_dir
        self.transform = transform
        self.clip_model = clip_model
        self.preprocess = preprocess
        self.device = device
        self.image_files = [f for f in os.listdir(os.path.join(root_dir, "original")) if (f.endswith('.png') and f[0]!='.')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        original_img_path = os.path.join(self.root_dir, "original", img_name)
        expert_img_path = os.path.join(self.root_dir, "expert", img_name)

        original_image = Image.open(original_img_path).convert("RGB")
        expert_image = Image.open(expert_img_path).convert("RGB")

        if self.transform:
            original_image = self.transform(original_image)
            expert_image = self.transform(expert_image)
        diff = original_image - expert_image
        mean_diff = diff.view(-1, 3).mean(dim=0).view(1, 3, 1, 1)  # Calculate mean per channel and reshape
        original_features = None

        if self.clip_model and self.preprocess:
            original_features = get_clip_features(original_image, self.preprocess, self.clip_model, self.device)

        return original_image, expert_image, mean_diff, original_features


In [ ]:

# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor()
# ])

# clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
# preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# dataset = MIT5KDataset(root_dir='train', transform=transform, clip_model=clip_model, preprocess=preprocess, device=device)
# dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


# GAN Test

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, feature_dim=512):
        super(Generator, self).__init__()
        # Output three values, one for each channel in RGB
        self.fc = nn.Linear(feature_dim, 3)  # Only three outputs for RGB

    def forward(self, features):
        # Generate one RGB value per image
        rgb_values = self.fc(features)  # shape: [batch_size, 3]
        rgb_values = torch.sigmoid(rgb_values)  # Normalize to range [0, 1]

        # Expand RGB values across the spatial dimensions (e.g., 256x256)
        output_size = (features.shape[0], 3, 256, 256)  # Define the desired output size
        img = rgb_values.view(features.shape[0], 3, 1, 1)
        img = img.expand(output_size)  # Expand the single RGB value to fill the image size

        return img


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(6, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 1, 4, padding=0),
            nn.AdaptiveAvgPool2d(1),
            nn.Sigmoid()
        )

    def forward(self, img_a, img_b):
        img_input = torch.cat((img_a, img_b), 1)
        validity = self.model(img_input)
        return validity.view(-1, 1)


In [ ]:

generator = Generator().to(device)
discriminator = Discriminator().to(device)

adversarial_loss = nn.BCELoss()
loss_fn = nn.MSELoss()

optimizer_G = Adam(generator.parameters(), lr=0.0001)
optimizer_D = Adam(discriminator.parameters(), lr=0.001)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

dataset = MIT5KDataset(root_dir='train', transform=transform, clip_model=clip_model, preprocess=preprocess, device=device)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
!rm -rf images
!mkdir images

In [ ]:
from tqdm.notebook import tqdm
import torch
from torchvision.utils import save_image

num_epochs = 1
print_interval = 100

for epoch in tqdm(range(num_epochs), desc='Epochs'):
    for i, (imgs, experts, diff, clip_features) in enumerate(tqdm(dataloader, desc='Batches', leave=False)):
        valid = torch.ones(imgs.size(0), 1, device=device, dtype=torch.float32)
        fake = torch.zeros(imgs.size(0), 1, device=device, dtype=torch.float32)

        real_imgs = imgs.to(device)
        expert_imgs = experts.to(device)
        clip_features = clip_features.to(device)

        optimizer_D.zero_grad()

        # Train Discriminator with real and fake data
        # Pass the separate images directly to the discriminator
        real_loss = adversarial_loss(discriminator(real_imgs, expert_imgs), valid)
        gen_filter = generator(clip_features)
        synthetic_expert = real_imgs + gen_filter
        fake_loss = adversarial_loss(discriminator(real_imgs, synthetic_expert.detach()), fake)

        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train Generator to fool Discriminator
        # Pass the separate images directly to the discriminator
        g_loss = adversarial_loss(discriminator(real_imgs, synthetic_expert), valid)
        g_loss.backward()
        optimizer_G.step()

        if i % print_interval == 0:
            tqdm.write(f"[Epoch {epoch+1}/{num_epochs}] [Batch {i+1}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

            save_image(synthetic_expert.data, f"/content/images/{epoch}_{i}_synthetic_expert.png", nrow=1, normalize=True)
            save_image(expert_imgs.data, f"/content/images/{epoch}_{i}_expert.png", nrow=1, normalize=True)
            save_image(real_imgs.data, f"/content/images/{epoch}_{i}_original.png", nrow=1, normalize=True)


Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3999 [00:00<?, ?it/s]

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


[Epoch 1/1] [Batch 1/3999] [D loss: 0.7076947093009949] [G loss: 0.707656979560852]
[Epoch 1/1] [Batch 101/3999] [D loss: 0.2099851369857788] [G loss: 1.3714221715927124]
[Epoch 1/1] [Batch 201/3999] [D loss: 0.10697304457426071] [G loss: 2.9840340614318848]
[Epoch 1/1] [Batch 301/3999] [D loss: 0.010047802701592445] [G loss: 4.259679317474365]
[Epoch 1/1] [Batch 401/3999] [D loss: 0.009326599538326263] [G loss: 4.473228454589844]
[Epoch 1/1] [Batch 501/3999] [D loss: 0.008939722552895546] [G loss: 4.52060604095459]
[Epoch 1/1] [Batch 601/3999] [D loss: 0.006883087567985058] [G loss: 4.955076694488525]
[Epoch 1/1] [Batch 701/3999] [D loss: 0.002522143069654703] [G loss: 5.501633644104004]
[Epoch 1/1] [Batch 801/3999] [D loss: 0.002988614607602358] [G loss: 5.647335052490234]
[Epoch 1/1] [Batch 901/3999] [D loss: 0.0024530678056180477] [G loss: 5.645647048950195]
[Epoch 1/1] [Batch 1001/3999] [D loss: 0.07767174392938614] [G loss: 3.347837448120117]
[Epoch 1/1] [Batch 1101/3999] [D loss

In [ ]:
# Save the Generator and Discriminator models
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')


## dev test

In [ ]:
# Load the models for testing
generator.load_state_dict(torch.load('generator.pth'))
discriminator.load_state_dict(torch.load('discriminator.pth'))
generator.eval()  # Set to evaluation mode
discriminator.eval()  # Set to evaluation mode


In [ ]:
dev_dataset = MIT5KDataset(root_dir='dev', transform=transform, clip_model=clip_model, preprocess=preprocess, device=device)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False)

# Testing loop
with torch.no_grad():
    for i, (test_imgs, test_experts, _, test_clip_features) in enumerate(dev_dataloader):
        dev_real_imgs = dev_imgs.to(device)
        dev_clip_features = dev_clip_features.to(device)
        dev_gen_filter = generator(dev_clip_features)
        dev_synthetic_expert = dev_real_imgs + test_gen_filter

        # Save dev images
        save_image(dev_synthetic_expert.data, f"/content/images/dev_{i}_synthetic_expert.png", nrow=1, normalize=True)
        save_image(dev_experts.data, f"/content/images/dev_{i}_expert.png", nrow=1, normalize=True)
        save_image(dev_real_imgs.data, f"/content/images/dev_{i}_original.png", nrow=1, normalize=True)


# CNN Test

In [ ]:
# reference: https://www.hackersrealm.net/post/extract-features-from-image-python
# reference: https://github.com/yuukicammy/mit-adobe-fivek-dataset
#from torch.utils.data.dataloader import DataLoader
#from dataset.fivek import MITAboveFiveK
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2D, MaxPooling2D, Flatten, Input, Conv2DTranspose
import numpy as np
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import tensorflow
import matplotlib.pyplot as plt
import os
import cv2

MODEL_SAVE_PATH = 'model/'
DEV_IMAGE_DIR = 'dev/original/'

def CNN_model():
  model = Sequential([
      Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
      MaxPooling2D((2,2)),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D((2,2)),
      Conv2D(64, (3, 3), activation='relu'),
      Flatten(),
      Dense(64, activation='relu'),
      Reshape((224, 224))
  ])
  return model

def fully_connected_model():
    model = Sequential([
       Dense(64, activation='relu', input_shape=(224, 224)),
       Dense(32, activation='relu'),
       Dense(4096, activation='linear'),
       Reshape((224,224))
    ])
    return model

def CNN_model_2():
    input_tensor = Input(shape=(224, 224, 3))
    conv2D1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
    conv2D2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2D1)

    tranpose1 = Conv2DTranspose(64, (3, 3), strides=(1, 1), activation='relu', padding='same')(conv2D2)
    output_tensor = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(tranpose1)
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

In [ ]:
# read from train dataset
path_edited = "train/expert/"
path_original = "train/original/"
VGG_model = VGG16()
VGG_model = Model(inputs=VGG_model.inputs, outputs=VGG_model.layers[-2].output)

CLIP_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
CLIP_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
emotion_list = ["amusement", "awe", "contentment", "excitement", "anger", "disgust", "fear", "sadness"]

# {"amusement":[value1, value2], "awe":[value1, value2]}
color_shift_values = {}
original_images_dict = {}
# initialize the color_shift_values
for emotion in emotion_list:
    color_shift_values[emotion] = []
    original_images_dict[emotion] = []
edited_list = os.listdir(path_edited)
original_list = os.listdir(path_original)
num = len(original_list)
num_epochs = 1
for i in tqdm(range(num)):
    original_img = cv2.imread(path_original+original_list[i])
    original_img = cv2.resize(original_img, (224, 224))
    # classification using CLIP
    inputs_clip = CLIP_processor(text=["image evokes amusement", "image evokes awe", "image evokes contentment", "image evokes excitement", "image evokes anger", "image evokes disgust", "image evokes fear", "image evokes sadness"], images=original_img, return_tensors="pt", padding=True)
    outputs_clip = CLIP_model(**inputs_clip)
    logits_per_image = outputs_clip.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    # get the highest prob in probs list
    highest_prob = torch.max(probs)
    highest_prob_index = 0
    #print(probs)
    for i in range(8):
        if (probs[0][i].item()==highest_prob):
            highest_prob_index = i
    image_class = emotion_list[highest_prob_index]
    original_images_dict[image_class].append(original_img)

    edited_img = cv2.imread(path_edited+edited_list[i])
    edited_img = cv2.resize(edited_img, (224, 224))
    #original_img = img_to_array(original_img)
    #edited_img = img_to_array(edited_img)
    original_color = cv2.cvtColor(original_img, cv2.COLOR_BGR2LAB)
    #print("original_color")
    #print(original_color)
    edited_color = cv2.cvtColor(edited_img, cv2.COLOR_BGR2LAB)
    #print("edited_color")
    #print(edited_color)
    # get color_shift_value
    color_diff = cv2.subtract(edited_color, original_color)
    color_shift_values[image_class].append(color_diff)

model_dict = {}
for emotion in emotion_list:
    if(len(original_images_dict[emotion])!=0):
        #print("len")
        #print(len(original_images_dict[emotion]))
        X = np.array(original_images_dict[emotion])
        print(X.shape)
        #print(X)
        y = np.array(color_shift_values[emotion])
        print(y.shape)
        #print(y)

        # build model with fully connected layer
        model_dict[emotion] = CNN_model_2()

        # print(model_dict[emotion].summary())
        model_dict[emotion].compile(optimizer='adam', loss='mean_squared_error')
        model_dict[emotion].fit(X, y, epochs=1, batch_size=num_epochs)

for emotion, model in model_dict.items():
    model.save(f'model/{emotion}.h5')  # Save each model in an H5 file

## dev test

In [ ]:
from tensorflow.keras.models import load_model as keras_load_model

def load_emotion_models(emotion_list, model_dir='model/'):

    model_dict = {}
    for emotion in emotion_list:
        try:
            model_path = os.path.join(model_dir, f'{emotion}.h5')
            model_dict[emotion] = keras_load_model(model_path)
            print(f"Loaded model for {emotion} from {model_path}.")
        except Exception as e:
            print(f"Error loading model for {emotion}: {e}")
    return model_dict


In [ ]:
DEV_IMAGE_DIR = 'dev/original/'
GEN_IMAGE_DIR = 'dev/gen/'

model_dict = load_emotion_models(emotion_list)

# Ensure the generated images directory exists
if not os.path.exists(GEN_IMAGE_DIR):
    os.makedirs(GEN_IMAGE_DIR)

# Process each image in the dev/original directory
for filename in os.listdir(DEV_IMAGE_DIR):
    file_path = os.path.join(DEV_IMAGE_DIR, filename)
    img = cv2.imread(file_path)
    img_resized = cv2.resize(img, (224, 224))

    # Classify image using CLIP model
    inputs_clip = CLIP_processor(text=[f"image evokes {emotion}" for emotion in emotion_list], images=img_resized, return_tensors="pt", padding=True)
    outputs_clip = CLIP_model(**inputs_clip)
    logits_per_image = outputs_clip.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    highest_prob_index = probs.argmax().item()
    image_class = emotion_list[highest_prob_index]

    # Use the corresponding model to generate the modified image
    if image_class in model_dict:
        img_transformed = model_dict[image_class].predict(np.expand_dims(img_resized, axis=0))[0]
        # Convert back to BGR for saving with OpenCV
        img_transformed = cv2.cvtColor(img_transformed, cv2.COLOR_RGB2BGR)
        output_path = os.path.join(GEN_IMAGE_DIR, filename)
        cv2.imwrite(output_path, img_transformed * 255)  # Assuming sigmoid activation in the last layer

print("Image processing completed. Modified images are saved in 'dev/gen'.")

# Evaluation

In [ ]:
!pip install kornia


In [ ]:
from typing import List
import numbers

import kornia as K
from kornia.geometry import resize
import numpy as np
import torch
import math
from kornia.filters import get_gaussian_kernel2d
from kornia.metrics import ssim, psnr
from torch import nn
import torch.nn.functional as F
from scipy.ndimage import gaussian_filter
from scipy.signal import get_window
from skimage.metrics import structural_similarity as scipy_ssim

In [ ]:
gen_dir = 'dev/gen/'
expert_dir = 'dev/expert/'

gen_images = sorted(os.listdir(gen_dir))
expert_images = sorted(os.listdir(expert_dir))
total_kornia_ssim, total_scipy_ssim, total_kornia_psnr = 0.0, 0.0, 0.0
new_size = (224, 224)
for gen_img, exp_img in zip(gen_images, expert_images):
    gen_path = os.path.join(gen_dir, gen_img)
    exp_path = os.path.join(expert_dir, exp_img)

    origin = resize(K.io.load_image(exp_path, K.io.ImageLoadType.RGB32)[None, ...], new_size)
    adjusted = resize(K.io.load_image(gen_path, K.io.ImageLoadType.RGB32)[None, ...], new_size)

    img1_np = np.moveaxis(origin.cpu().detach().numpy().squeeze(), 0, 2)
    img2_np = np.moveaxis(adjusted.cpu().detach().numpy().squeeze(), 0, 2)

    # Compute SSIM and PSNR
    k_ssim_value = torch.mean(ssim(origin, adjusted, window_size=11)).item()
    sc_ssim_value = scipy_ssim(im1=img1_np, im2=img2_np, gaussian_weights=True, data_range=1.0, use_sample_covariance=False, channel_axis=2)
    k_psnr_value = psnr(origin, adjusted, max_val=2.).item()

    total_kornia_ssim += k_ssim_value
    total_scipy_ssim += sc_ssim_value
    total_kornia_psnr += k_psnr_value

avg_kornia_ssim = total_kornia_ssim / 500
avg_scipy_ssim = total_scipy_ssim / 500
avg_kornia_psnr = total_kornia_psnr / 500

print(f"kornia ssim: {avg_kornia_ssim}")
print(f"scipy ssim: {avg_scipy_ssim}")


kornia ssim: 0.756060779094696
scipy ssim:  0.7624991536140442


In [ ]:
print(f"kornia psnr: {avg_kornia_psnr}")

kornia psnr: 26.36754035949707


# Test Data Generation

In [ ]:
import os
import cv2
import numpy as np
import torch

TEST_IMAGE_DIR = 'test/original/'
GENERATED_IMAGE_DIR = 'test/gen/'

if not os.path.exists(GENERATED_IMAGE_DIR):
    os.makedirs(GENERATED_IMAGE_DIR)

for filename in os.listdir(TEST_IMAGE_DIR):
    file_path = os.path.join(TEST_IMAGE_DIR, filename)
    img = cv2.imread(file_path)
    if img is None:
        continue
    img_resized = cv2.resize(img, (224, 224))

    inputs_clip = CLIP_processor(text=[f"image evokes {emotion}" for emotion in emotion_list], images=img_resized, return_tensors="pt", padding=True)
    outputs_clip = CLIP_model(**inputs_clip)
    logits_per_image = outputs_clip.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    highest_prob_index = probs.argmax().item()
    image_class = emotion_list[highest_prob_index]

    # Predict the transformation using the appropriate model
    if image_class in model_dict:
        img_transformed = model_dict[image_class].predict(np.expand_dims(img_resized, axis=0))[0]
        img_transformed = (img_transformed * 255).astype(np.uint8)  # Scale output to 0-255 range for saving
        output_path = os.path.join(GENERATED_IMAGE_DIR, filename)
        cv2.imwrite(output_path, img_transformed)
        print(f"Processed and saved: {output_path}")

print("All images have been processed and saved in the test/gen directory.")
